In [1]:
import sys, os
sys.path.insert(0, '../')
sys.path.insert(0, '../python_src/')

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns

import homology
import chomology

In [ ]:

# NV = 3
# NE = 3

# DIM = 2

# edgei = [0, 0, 1]
# edgej = [1, 2, 2]

# L = np.array([10.0, 10.0], float)

# vert_pos = np.zeros(DIM*NV, float)
# vert_pos[0:DIM] = [0, 1]
# vert_pos[DIM:2*DIM] = [1, 1]
# vert_pos[2*DIM:3*DIM] = [0, 0]

# corner_strains = np.array([0, 3, 5])

NV = 5
NE = 6

DIM = 2

edgei = [0, 0, 1, 2, 1, 3]
edgej = [1, 2, 3, 3, 4, 4]

L = np.array([10.0, 10.0], float)

vert_pos = np.zeros(DIM*NV, float)
vert_pos[0:DIM] = [0, 1]
vert_pos[DIM:2*DIM] = [1, 1]
vert_pos[2*DIM:3*DIM] = [0, 0]
vert_pos[3*DIM:4*DIM] = [1, 0]
vert_pos[4*DIM:5*DIM] = [2.5, 0.5]

corner_strains = np.array([0, 5, 3, 2, 6, 4, 1])


print("pos", vert_pos)
    
graph = chomology.Graph(NV, NE, edgei, edgej)
graph.set_embedding(DIM, vert_pos, L)

corners = chomology.find_corners(graph)

print(corners)

# vertex_time = chomology.perform_corner_transform(corners, corner_strains, graph, True)

# print(vertex_time)

comp = chomology.construct_corner_complex(corners, graph)

print("Corner Complex:")

# for i in range(comp.ncells):
#     print(i, comp.get_label(i), comp.get_dim(i), comp.get_facets(i), comp.get_cofacets(i))

subcomplex_order = chomology.perform_watershed_transform(corner_strains, comp, ascend=True, co=True)

print(subcomplex_order)

filt_asc = chomology.construct_filtration(corner_strains, subcomplex_order, comp, ascend=True, co=True)
filt_desc = chomology.construct_filtration(corner_strains, subcomplex_order, comp, ascend=False, co=True)

for i in range(comp.ncells):
    print(i, filt_asc.get_time(i), filt_asc.get_subcomplex_order(i))

print("Corner Filtration")
print("Ascending", filt_asc.get_filtration())
print("Descending", filt_desc.get_filtration())

graph_comp = chomology.construct_graph_complex(graph, False)

filt_asc = chomology.reduce_filtration(filt_asc, comp, graph_comp)
filt_desc = chomology.reduce_filtration(filt_desc, comp, graph_comp)

for i in range(graph_comp.ncells):
    print(i, filt_asc.get_time(i), filt_asc.get_subcomplex_order(i))

print("Graph Filtration")
print("Ascending", filt_asc.get_filtration())
print("Descending", filt_desc.get_filtration())


# check if ascending filtration on -corner_strains is same as descending filtration 

In [ ]:
subcomplex_order = chomology.perform_watershed_transform(-np.array(corner_strains), comp, ascend=True, co=True)
filt_asc = chomology.construct_filtration(-np.array(corner_strains), subcomplex_order, comp, ascend=True, co=True)
print(filt_asc.get_filtration())

subcomplex_order = chomology.perform_watershed_transform(corner_strains, comp, ascend=True, co=True)
filt_desc = chomology.construct_filtration(corner_strains, subcomplex_order, comp, ascend=False, co=True)
print(filt_desc.get_filtration())


In [ ]:
print("Calculating persistence pairs...")

pairs = chomology.calc_extended_persistence(filt_asc, filt_desc, graph_comp)


print("Number Pairs:", len(pairs[0])+len(pairs[1])+len(pairs[2]))
     
print(pairs)

print("Ascending")
for (i, j) in pairs[0]:
    print(graph_comp.get_dim(i), graph_comp.get_dim(j), i, j, filt_asc.get_time(i), filt_asc.get_time(j))

print("Descending")
for (i, j) in pairs[1]:
    print(graph_comp.get_dim(i), graph_comp.get_dim(j), i, j, filt_desc.get_time(i), filt_desc.get_time(j))
    
print("Complete")

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'cm'
sns.set(color_codes=True, palette='deep')
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})

sns.set_context('talk', font_scale=1.5)

for d in [0, 1]:

    fig = plt.figure(figsize=(8, 8))

    ax1 = fig.add_subplot(1,1,1)

    birth = []
    death = []
    for (i, j) in pairs[0]:
        if comp.get_dim(i) == d:
            birth.append(filt_asc.get_time(i))
            death.append(filt_asc.get_time(j))

    ax1.scatter(birth, death, marker='o', color='b', label="Ordinary")

    birth = []
    death = []
    for (i, j) in pairs[1]:
        if comp.get_dim(i) == d:
            birth.append(filt_desc.get_time(i))
            death.append(filt_desc.get_time(j))

    ax1.scatter(birth, death, marker='v', color='r', label="Relative")

    birth = []
    death = []
    for (i, j) in pairs[2]:
        if comp.get_dim(i) == d:
            birth.append(filt_asc.get_time(i))
            death.append(filt_desc.get_time(j))

    ax1.scatter(birth, death, marker='^', color='g', label="Extended")


    ax1.plot(np.linspace(np.min(corner_strains), np.max(corner_strains), 100), 
             np.linspace(np.min(corner_strains), np.max(corner_strains), 100), 'k--')

    ax1.set_title(r"$d={}$".format(d))

    ax1.set_xlabel(r"Birth - Strain Magnitude $\varepsilon^2$")
    ax1.set_ylabel(r"Death - Strain Magnitude $\varepsilon^2$")

    # ax1.set_xscale('log')
    # ax1.set_yscale('log')

    ax1.legend(fontsize='medium')

    plt.tight_layout()

    plt.show()






# TIME = 0

# birth = [[] for i in range(mcomp.dim+1)]
# persistence = [[] for i in range(mcomp.dim+1)]

# for (i, j) in pairs:
    
#     if j is None or rfilt.get_time(i) == rfilt.get_time(j):
#         continue
        
    
    
#     d = graph_comp.get_dim(i)
#     birth[d].append(rfilt.get_time(i))
#     persistence[d].append(rfilt.get_time(j)-rfilt.get_time(i))

# fig = plt.figure(figsize=(8,8))
        
# ax1 = fig.add_subplot(1,1,1)

# ax1.set_yscale('log')

# ax1.scatter(birth[0], persistence[0], marker='.', color='b', label="$0$-cycles")
# ax1.scatter(birth[1], persistence[1], marker='.', color='r', label="$1$-cycles")

# ax1.set_xlabel(r"Birth - Strain Magnitude $\varepsilon^2$")
# ax1.set_ylabel(r"Persistence - Strain Magnitude $\varepsilon^2$")
# # ax1.set_ylim(1e-8, 1e0)

# # ax1.hlines(10**(-1.9), -0.4, 1.0)

# # ax1.legend(fontsize='large')

# plt.tight_layout()

# plt.show()
